In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as pf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv2D, Dropout, MaxPooling2D

Using TensorFlow backend.


In [2]:
from PIL import Image

dir = 'train'
new_dir = 'train_128'
images = []
filenames = []

for filename in os.listdir(dir):
    filenames.append(filename[:-4])
    img = Image.open(dir + '/' + filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    images.append(np.array(img))
    #img.save(new_dir + '/' + filename)

images = np.array(images)
filenames = np.array(filenames)

In [3]:
labels = pd.read_csv('labels.csv').set_index('id')

filenames = pd.DataFrame(filenames)
filenames.columns = ['id']
filenames = filenames.set_index('id')
filenames = filenames.join(labels, on=filenames.index)

In [12]:
X = images/255
y = np.array(filenames.breed)
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
num_breed = len(set(filenames.breed))

def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, input_shape=(128,128,3), kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    
    #model.add(Dense(128, activation='relu'))
    model.add(Dense(num_breed, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
model = baseline_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 127, 127, 32)      416       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 62, 62, 32)        4128      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 30752)             0         
__________

In [15]:
X_train.shape, y_train.shape

((8177, 128, 128, 3), (8177, 120))

In [17]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=20)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 15s 2ms/step - loss: 4.7519 - acc: 0.0223 - val_loss: 4.5784 - val_acc: 0.0347
Epoch 2/10
8177/8177 [==============================] - 15s 2ms/step - loss: 3.9791 - acc: 0.1240 - val_loss: 4.6575 - val_acc: 0.0460
Epoch 3/10
8177/8177 [==============================] - 15s 2ms/step - loss: 2.2394 - acc: 0.4740 - val_loss: 5.5976 - val_acc: 0.0455
Epoch 4/10
8177/8177 [==============================] - 15s 2ms/step - loss: 0.6952 - acc: 0.8347 - val_loss: 7.1739 - val_acc: 0.0425
Epoch 5/10
8177/8177 [==============================] - 15s 2ms/step - loss: 0.1945 - acc: 0.9593 - val_loss: 8.3632 - val_acc: 0.0513
Epoch 6/10
8177/8177 [==============================] - 15s 2ms/step - loss: 0.0703 - acc: 0.9874 - val_loss: 9.0679 - val_acc: 0.0425
Epoch 7/10
8160/8177 [============================>.] - ETA: 0s - loss: 0.0418 - acc: 0.9920

KeyboardInterrupt: 